In [1]:
import scipy.optimize as optimize

import pandas_datareader as pdr
import datetime 
import pandas as pd
import numpy as np

import pandas_datareader.data as web
import requests_cache
import matplotlib.pyplot as plt
from datetime import date

# Constants

# Current TBILL rate
TBILL_PERCENT = 1.66

In [2]:
from prefutils import *

# initialize web cache
SESSION = init_fetch_session()

# read the FloatingReset database file
floats = pd.read_csv('./floats.csv')

# filter some tickers, remove some columns
floats = filter_floating_reset_list(floats)


In [3]:
tprefs = update_data_frame_with_prices_and_drop_reference(floats,SESSION)

In [4]:
tprefs = update_dataframe_with_market_spread(tprefs, TBILL_PERCENT)

In [5]:
#tpm = update_dataframe_with_rating_averages(tprefs)
#tpm.head(5)

In [6]:
# market spread deltas: model increasing or decreasing levels of fear
SPREAD_DELTAS = [ 0.2, 0, -0.1, -.3]

# define 4 scenarios, with corresponding probabilities and names.
# Name: {tbillrate, number of years, probability}
TBILL_SCN = { 
    "Constant" :  (1.66, 1, 0.10),
    "SlightDrop": (1.40, 1, 0.40),
    "BigDrop":    (1.10, 1, 0.30) ,
    "Panic":      (0.70, 1, 0.20)}


foo = create_tbill_scenarios_per_market_spreads(tprefs, TBILL_SCN, SPREAD_DELTAS)
foo.head(3)


,Ticker,Rating,Spread,Price,MSpread,EffMSpread,MSpread_Delta,Expected_Gain,Constant,PriceConstant,SlightDrop,PriceSlightDrop,BigDrop,PriceBigDrop,Panic,PricePanic
0,AZP.PR.C,P5H,418.0,18.71,6.1433,6.3433,0.2,0.023997,0.053045,18.242475,0.037444,18.015575,0.018390,17.734070,-0.009010,17.321426
1,AIM.PR.B,P4H,375.0,16.85,6.3667,6.5667,0.2,0.017887,0.055957,16.440371,0.035520,16.161020,0.010543,15.815149,-0.025397,15.309563
2,BPO.PR.S,P3I,348.0,19.30,4.9980,5.1980,0.2,0.007077,0.037422,18.737241,0.021267,18.490452,0.001315,18.180375,-0.027831,17.717870


In [7]:

ranked = do_the_ranking(foo,TBILL_SCN,10)
ranked.head(3)

,Ticker,TotalRankSum,AvgExpectedGain
2,AZP.PR.C,16,0.057293
24,SLF.PR.K,16,0.054615
0,AIM.PR.B,16,0.049904


In [8]:
ranked = ranked.merge(floats[['Ticker','Rating','MSpread']],how='inner',on='Ticker')
ranked.head(10)

,Ticker,TotalRankSum,AvgExpectedGain,Rating,MSpread
0,AZP.PR.C,16,0.057293,P5H,6.1433
1,SLF.PR.K,16,0.054615,P1L,4.0225
2,AIM.PR.B,16,0.049904,P4H,6.3667
3,BPO.PR.S,16,0.046505,P3I,4.9980
4,FFH.PR.D,12,0.039586,P3I,4.8860
5,FFH.PR.J,16,0.031697,P3I,4.9490
6,IFC.PR.D,9,0.030795,P2I,4.1778
7,BNS.PR.F,8,0.028129,P2Hb,1.4087
8,BRF.PR.B,6,0.026354,P3H,4.6714
9,ALA.PR.B,9,0.025580,P3L,5.1625


In [9]:
# Now, for hedging. For each security, it needs to make a 5% hurdle.
# So for example, if IFC.PR.D is only expected to make 3.5%, then it needs a 1.5% hedge gain to compensate.

def sumproduct_scenario(scn) :
    sum = 0;
    for (a,_,b) in scn.values() : sum += a*b       
    return sum

expected_tbill_rate = sumproduct_scenario(TBILL_SCN)
# print(expected_tbill_rate)
expected_drop = TBILL_PERCENT - expected_tbill_rate
# So if we drop by 0.4%, we want a 1.5% hedge gain then hedge ratio is 1.5/0.4 = 3.75. Would have to
# hedge $3.75 for each $ of IFC.PR.D
expected_drop

# but actually, should we reduce the amount of net hedges based on shares that exceed the hurdle?



0.46399999999999997

In [10]:
GAIN_HURDLE=0.05
ranked['GainToHedgePercent'] = [max(GAIN_HURDLE-x,0)*100 for x in ranked['AvgExpectedGain']]
ranked['HedgeRatio'] = [x/expected_drop for x in ranked['GainToHedgePercent']]
ranked.head(15)

,Ticker,TotalRankSum,AvgExpectedGain,Rating,MSpread,GainToHedgePercent,HedgeRatio
0,AZP.PR.C,16,0.057293,P5H,6.1433,0.000000,0.000000
1,SLF.PR.K,16,0.054615,P1L,4.0225,0.000000,0.000000
2,AIM.PR.B,16,0.049904,P4H,6.3667,0.009561,0.020606
3,BPO.PR.S,16,0.046505,P3I,4.9980,0.349471,0.753171
4,FFH.PR.D,12,0.039586,P3I,4.8860,1.041418,2.244435
5,FFH.PR.J,16,0.031697,P3I,4.9490,1.830350,3.944719
6,IFC.PR.D,9,0.030795,P2I,4.1778,1.920503,4.139014
7,BNS.PR.F,8,0.028129,P2Hb,1.4087,2.187063,4.713497
8,BRF.PR.B,6,0.026354,P3H,4.6714,2.364635,5.096195
9,ALA.PR.B,9,0.025580,P3L,5.1625,2.441968,5.262862
